In [1]:
import numpy as np
import math

import scipy as sp
from scipy.spatial.transform import Rotation
from scipy.spatial import distance
from scipy.spatial import KDTree

import time

import os
import csv
import pandas as pd

import joblib

## Raup's model
Calculate the following values based on raup's model.
- Coordinates of generating curve
- Coordinates of center of generating curve
- Frenet frame

In [2]:
def RaupModel(W_r, T_r, D_r, S_r, delta_r, gamma_r, r0, theta, phi, a = 1):
    """Raup's model
    
    Calculate the coordinates of the shell surface（x, y, z）.
    
    Args:
        W_r: the whorl expansion rate
        T_r: the rate of helicocone translation 
        D_r: the position of the generating curve in relation to the coiling axis
        S_r: the aspect ratio of the generating curve
        delta_r: angle between the generating curve and the coiling axis
        gamma_r: angle between the generating curve and the shell radius
        r0: initial radius
        theta: coiling angle
        phi: angle of rotation about aperture

    Returns:
        X, Y, Z: coordinates of generating curve
        px, py, pz: coordinates of center of generating curve
        xi1, xi2, xi3: Frenet frame
    
    """
    
    w = W_r**(theta/(2*np.pi))
    px = w * (2*D_r/(1 - D_r) + 1 )*np.cos(theta)
    py = w * (2*D_r/(1 - D_r) + 1 )*np.sin(theta)
    pz = w * (2*T_r*(D_r/(1 - D_r) + 1) )
    
    theta_grid, phi_grid = np.meshgrid(theta, phi)
    
    ww = W_r**(theta_grid/(2*np.pi))

    X =r0*ww*(
        (a*np.sin(theta_grid)*(-S_r*np.cos(phi_grid)*np.sin(gamma_r)+np.cos(gamma_r)*np.sin(delta_r)*np.sin(phi_grid)))/S_r+(np.cos(theta_grid))*(1-2*D_r/(-1+D_r)+a*np.cos(gamma_r)*np.cos(phi_grid)+a*np.sin(gamma_r)*np.sin(delta_r)*np.sin(phi_grid)/S_r)
    )
    
    Y =r0*ww*(
        np.cos(theta_grid)*(a*np.cos(phi_grid)*np.sin(gamma_r)
                       -a*np.cos(gamma_r)*np.sin(delta_r)*np.sin(phi_grid)/S_r)
        +np.sin(theta_grid)*(1-2*D_r/(-1+D_r)+a*np.cos(gamma_r)*np.cos(phi_grid)
                        +a*np.sin(gamma_r)*np.sin(delta_r)*np.sin(phi_grid)/S_r)
    )
    Z =r0*ww * (-2*T_r/(-1+D_r) + a*np.cos(delta_r)*np.sin(phi_grid)/S_r)
    
    r=r0*w
    
    ##
    ##Frenet frame
    ##
    logw = np.log(W_r)
    xi1x=(
        (1+D_r)*(np.cos(theta)*logw-2*np.pi*np.sin(theta))
    )/np.sqrt(4*T_r**2*logw**2+(1+D_r)**2*(4*(np.pi)**2+(logw)**2))
    xi1y=(
        (1+D_r)*(2*np.pi*np.cos(theta)+logw*np.sin(theta))
    )/np.sqrt(4*T_r**2*logw**2+(1+D_r)**2*(4*(np.pi)**2+logw**2))
    xi1z=np.tile((2*T_r*logw)/np.sqrt(4*T_r**2*logw**2+(1+D_r)**2*(4*(np.pi)**2+logw**2)),len(theta))
    
    xi2x=-(2*np.pi*np.cos(theta)+logw*np.sin(theta))/np.sqrt(4*np.pi**2+logw**2)
    xi2y=(np.cos(theta)*logw-2*np.pi*np.sin(theta))/np.sqrt(4*np.pi**2+logw**2)
    xi2z=np.zeros(len(theta))
    
    
    xi1=np.array([xi1x,xi1y,xi1z]).transpose()
    xi2=np.array([xi2x,xi2y,xi2z]).transpose()
    xi1 = np.apply_along_axis(normalize, 1, xi1)
    xi2 = np.apply_along_axis(normalize, 1, xi2)
    xi3 = np.cross(xi1, xi2)
    xi3 = np.apply_along_axis(normalize, 1, xi3)
    
    return (X, Y, Z, px, py, pz,r,xi1, xi2, xi3)

## Convert variable
- Convert raup's model parameters$W_{R}, T_{R}, D_{R}, \Delta, \Gamma $ to growing tube model parameters$E_{G}, C_{G}, T_{G}, \delta , \gamma $.
- Convert $\theta$ to $l$.

In [3]:
def cvt_Raup2GT(W_r, T_r, D_r, delta_r, gamma_r):
    
    E_g = ((1 - D_r)*np.log(W_r))/(np.sqrt(4*(np.pi**2)*(1+D_r)**2+((np.log(W_r))**2)*((1+D_r)**2+4*T_r**2)))
    C_g = ((2*np.pi*(1 - D_r**2))*np.sqrt(4*np.pi**2+(np.log(W_r)**2)))/(4*(np.pi**2)*(1+D_r)**2+((np.log(W_r))**2)*((1+D_r)**2+4*T_r**2))
    T_g = (4*np.pi*T_r*(1 - D_r)*np.log(W_r))/(4*(np.pi**2)*(1 + D_r)**2+((np.log(W_r))**2)*((1+D_r)**2+4*T_r**2))
    
    delta_g = np.arcsin(
        (2*T_r*np.cos(delta_r)*np.log(W_r)*(2*np.pi*np.cos(gamma_r) - np.log(W_r)*np.sin(gamma_r))
         -(1+D_r)*(4*np.pi**2+np.log(W_r)**2)*np.sin(delta_r))/np.sqrt((4*np.pi**2+np.log(W_r)**2)*(4*(1+D_r)**2*np.pi**2+((1+D_r)**2+4*T_r**2)*np.log(W_r)**2))
    )
    
    gamma_g_num = np.sqrt(2)*np.cos(delta_r)*(np.cos(delta_r)*np.log(W_r)
                                        +2*np.pi*np.sin(gamma_r))
    gamma_g_denom = np.sqrt(
        (4*np.cos(delta_r)**2 *
         (8*(1 + D_r)**2 * np.pi**4 + T_r**2 * np.log(W_r)**2 * (np.cos(2*gamma_r)*(-4*np.pi**2 + np.log(W_r)**2) + 4*np.pi*np.log(W_r)*np.sin(2*gamma_r)))
         +np.log(W_r)*(8*np.pi**2 * ((1+D_r)**2 + 3* T_r**2 + 3*T_r**2 + (1+D_r-T_r)*(1+D_r+T_r)*np.cos(2*delta_r))*np.log(W_r) + ((1+D_r)**2 + 6*T_r**2 + ((1+D_r)**2 - 2*T_r**2)*np.cos(2*delta_r))*np.log(W_r)**3 
                       + 4*(1+D_r)*T_r*(4*np.pi**2 + np.log(W_r)**2)*(2*np.pi*np.cos(gamma_r) - np.log(W_r)*np.sin(gamma_r))*np.sin(2*gamma_r)))
        /((4*(1+D_r)**2)*np.pi**2 + ((1+D_r)**2 + 4*T_r**2)*np.log(W_r)**2)
    )
    gamma_g = -np.arcsin(-gamma_g_num/gamma_g_denom)
    
    return E_g, C_g, T_g, delta_g, gamma_g

In [4]:
def cvt_theta2l(theta,W_r,T_r,D_r,r0):
    l=-np.sqrt(4*(1 + D_r)**2 * np.pi**2 + ((1 + D_r)**2 + 4* T_r**2)*np.log(W_r)**2)*r0*(-1+W_r**(theta/(2*np.pi)))/((-1+D_r)*np.log(W_r))
    return l

## Growing tube model
Calculate the following values based on the growing tube model
- Frenet frame($\xi_{1}i,\xi_{2}i,\xi_{3}i$) inclined at angles of $\delta$ and $\gamma$
- Coordinates of shell surface and soft body

In [5]:
def normalize(vec):
    norm = np.linalg.norm(vec)
    return vec/norm

def GTModel(E_g, C_g, T_g, delta_g, gamma_g, r0, l, dphi, x0, y0, z0, xi10, xi20, xi30, n, a=1):
    """Growing tube model
    Calculate the Frenet frame inclined at angles of delta_g and gamma_g
    
    Args:
        E_g: the growth rate at a certain stage s
        C_g: the curvature standardized by the radius r of the tube at growth stage s
        T_g: the tortuosity standardized by the radius r of the tube at growth stage s
        delta_g: angle around xi2
        gamma_g: angle around xi3
        r0: initial value of radius
        l: arc length represented by s
        dphi: width of angle phi
        x0, y0, z0: initial coordinates
        xi10, xi20, xi30: initial value of Frenet frame
    
    Returns:
        s: growth stage
        r: radius at growth stage s
        px, py, pz: coordinates of center of generating curve
        xi1, xi2, xi3: Frenet frame
        xi1_i, xi2_i, xi3_i: Frenet frame inclined at angles of delta_g and gamma_g
        
    """
    
    s=np.log(1+(E_g*l/r0))/E_g
    r =r0*np.exp(E_g*s)

    
    D = (C_g**2+T_g**2)**0.5
    ED3E2pD2 = E_g*D**3*(E_g**2 + D**2)
    expEs = np.exp(E_g*s)
    sinDs = np.sin(D*s)
    cosDs = np.cos(D*s)
    

    ##
    ## Growth Trajectory
    ##
    P = r0*D*(((D**2) * (T_g**2) + (E_g**2) * (T_g**2) + C_g**2 * E_g**2 * cosDs 
               + E_g*D*(C_g**2)*sinDs)*expEs - D**2 * (E_g**2 + T_g**2))/ED3E2pD2
    Q = r0*C_g*D*E_g*(-expEs*(C_g**2 + T_g**2)*cosDs 
                      + D*(D + expEs*E_g*sinDs))/ED3E2pD2
    R = r0*C_g*T_g*D*(((E_g**2) + (D**2) - (E_g**2) * cosDs - E_g*D*sinDs)*expEs - D**2)/ED3E2pD2

    X0 = np.array([x0, y0, z0])
    R0 = np.array([xi10, xi20, xi30]).transpose()

    px, py, pz = np.transpose(X0 + np.transpose(np.dot(R0, np.array([P, Q, R]))))

    ##
    ## Frenet frame
    ##
    xi1 = np.dot(
        R0,
        np.array([(T_g**2 + C_g**2*cosDs)/D**2, C_g*sinDs/D, C_g*T_g*(1 - cosDs)/D**2])
    ).transpose()
    xi2 = np.dot(
        R0,
        np.array([(-C_g*sinDs/D), cosDs, T_g*sinDs/D])
    ).transpose()
    xi3 = np.dot(
        R0,
        np.array([(C_g*T_g*(1 - cosDs))/D**2, (-T_g*sinDs)/D, (C_g**2+T_g**2*cosDs)/D**2])
    ).transpose()
    
    xi1 = np.apply_along_axis(normalize, 1, xi1)
    xi2 = np.apply_along_axis(normalize, 1, xi2)
    xi3 = np.apply_along_axis(normalize, 1, xi3)

    
    rot2 = Rotation.from_rotvec(delta_g*xi2).as_matrix()
    rot3 = Rotation.from_rotvec(gamma_g*xi3).as_matrix()
    rot_g = np.array([np.dot(rot3[i], rot2[i]) for i in range(len(rot2))])
    
    xi1_i  = np.array([normalize(np.dot(rot_g[i], xi1[i])) for i in range(len(xi1))])
    xi2_i  = np.array([normalize(np.dot(rot_g[i], xi2[i])) for i in range(len(xi2))])
    xi3_i  = np.array([normalize(np.dot(rot_g[i], xi3[i])) for i in range(len(xi3))])
    
    xi1_i = np.apply_along_axis(normalize, 1, xi1_i)
    xi2_i = np.apply_along_axis(normalize, 1, xi2_i)
    xi3_i = np.apply_along_axis(normalize, 1, xi3_i)

    return(s,r,px, py, pz, xi1, xi2, xi3,xi1_i,xi2_i,xi3_i)

In [6]:
def make_gencurves(s,r,px, py, pz,xi1_i,xi2_i,xi3_i,dphi,r0,E_g):
    
    gencurves = [] 

    s_theor=[]
    px_list=[]
    py_list=[]
    pz_list=[]
    r_list=[]
    
    for i in range(len(xi1_i)):
        phi=np.linspace(0,2*np.pi,int(np.ceil(2*np.pi*(r[i])/dphi)))
        rot1i = Rotation.from_rotvec(np.tensordot(phi, xi1_i[i],axes=0)).as_matrix()
        rr=[np.linspace(dphi,r0*np.exp(E_g*s[m]),int(np.ceil(r0*np.exp(E_g*s[m]))/dphi)) for m in range(0,len(s))]
        gencurve = [rr[i][k]*np.dot(rot1i[j], xi2_i[i]) for k in range(len(rr[i])) for j in range(len(rot1i))]
        gencurves.extend(gencurve)

        s_theor.extend(np.full(len(gencurve),s[i]))
        px_list.extend(np.full(len(gencurve),px[i]))
        py_list.extend(np.full(len(gencurve),py[i]))
        pz_list.extend(np.full(len(gencurve),pz[i]))
        r_list.extend(np.full(len(gencurve),r[i]))

    gencurves = np.array(gencurves)

    s_theor=np.array(s_theor)
    px=np.array(px_list)
    py=np.array(py_list)
    pz=np.array(pz_list)
    r=np.array(r_list)
    
    coords_theor=np.array([px, py, pz]).transpose() + gencurves

    return(coords_theor,s_theor,r,px, py, pz)

## Make voxel
Make voxel to store shell state

In [7]:
def vox_range(X,Y,Z):
    x_max=X.max()
    x_min=X.min()
    
    y_max=Y.max()
    y_min=Y.min()
    
    z_max=Z.max()
    z_min=Z.min()
    
    return(x_max,x_min,y_max,y_min,z_max,z_min)

In [8]:
def vox(x_max,x_min,y_max,y_min,z_max,z_min,vox_reso):
    """Discretize a theoretical model
    
    Returns:
        dx: width of the voxel
        vox_coords: 3D array of voxel center coordinates
        cords_list: 1D array of voxel center coordinates
        vox_state: 3D array to store voxel states
        
    """
    
    x_dist=np.abs(x_max-x_min)
    y_dist=np.abs(y_max-y_min)
    z_dist=np.abs(z_max-z_min)

    dx=np.abs(x_max-x_min)/vox_reso

    if z_dist/dx<25:
        dx=((x_dist*y_dist*z_dist)/(25*(vox_reso)**2))**(1/3)
    
    X,Y,Z=np.mgrid[x_min-dx:x_max+dx:dx,y_min-dx:y_max+dx:dx,z_min-2*dx:z_max+2*dx:dx]
    
    vox_coords=np.array([X,Y,Z])
    coords_list = np.array((X.flatten(), Y.flatten(), Z.flatten())).T
    vox_state=np.zeros((len(X[:,0,0]),len(Y[0,:,0]),len(Z[0,0,:])))
    vox_state_last=np.zeros((len(X[:,0,0]),len(Y[0,:,0]),len(Z[0,0,:])))
    
    print("vox_coords.shape=",vox_coords.shape)
    
    return(dx,vox_coords,coords_list,vox_state,vox_state_last)

In [9]:
def dsdist_calculation(xi2,r,px,py,pz,num):
    sr1=xi2[num]*(-r[num])+np.array([px[num],py[num],pz[num]])
    sr2=xi2[num-1]*(-r[num-1])+np.array([px[num-1],py[num-1],pz[num-1]])

    ds_dist=distance.euclidean(sr1,sr2)
    
    return(ds_dist)

## Prameter

In [10]:
num_w=10
_row=20
_col=20

Wr =10**(num_w*0.1)
Dr =0
gamr=0

deltarrange=np.linspace(-np.pi/2,np.pi/2,_row)
Trange=np.linspace(np.log10(0.1),np.log10(10),_col)

Tr,deltar=np.meshgrid(Trange,deltarrange)
ret=np.c_[10**(Tr.ravel()),deltar.ravel()]

Sr = 1
r0 = 1

theta_end=8*np.pi
theta = np.linspace(0, theta_end, 400)
Phi = np.linspace(0, 2*np.pi,100)
l_step0=800 

vox_reso=170

## Select a directory to save generated models

In [13]:
os.getcwd()

'C:\\Users\\araki\\Documents\\Araki_ms_2021\\modeldata_v3\\D0\\w10n400'

In [12]:
os.chdir('C:\\Users\\araki\\Documents\\Araki_ms_2021\\modeldata_v3\\D0\\w10n400')

## Generate model and voxelization

In [14]:
def generatemodel(theta,Wr,Tr,Dr,Sr,deltar,gamr,r0,Phi,l_step0,vox_reso,filename):
    
    l_end=cvt_theta2l(theta[-1],Wr,Tr, Dr, r0) 
    l_step=l_step0
    l=np.linspace(0,l_end,l_step)
    
    X, Y, Z, px, py, pz,r_r, xi1, xi2, xi3 = RaupModel(Wr, Tr, Dr, Sr, deltar, gamr, r0, theta, Phi)
    x_max,x_min,y_max,y_min,z_max,z_min=vox_range(X,Y,Z)
    dx,vox_coords,coords_list,vox_state,vox_state_last=vox(x_max,x_min,y_max,y_min,z_max,z_min,vox_reso)
    dphi=1/2*dx
    
    E_g, C_g, T_g, delta_g, gamma_g  = cvt_Raup2GT(Wr, Tr, Dr, deltar, gamr)
    s,r,px_g, py_g, pz_g, xi1_g, xi2_g, xi3_g,xi1_i,xi2_i,xi3_i=GTModel(E_g, C_g, T_g, delta_g, gamma_g, r0, l, dphi, px[0], py[0], pz[0], xi1[0], xi2[0], xi3[0],n=1,a=1)
    ds=dsdist_calculation(xi2_i,r,px_g,py_g,pz_g,num=-1)
    
    if deltar<0:
        if np.arccos(np.dot(xi1_g[0],[0,0,1]))<abs(deltar):
            TF=False
        else:    
            TF=True
    else:
        TF=True
        
    if TF==False:
        pass
    else:
        while ds>0.9*dx:
            l_step=l_step+50
            l=np.linspace(0,l_end,l_step)

            s,r,px_g, py_g, pz_g, xi1_g, xi2_g, xi3_g,xi1_i,xi2_i,xi3_i=GTModel(E_g, C_g, T_g, delta_g, gamma_g, r0, l, dphi, px[0], py[0], pz[0], xi1[0], xi2[0], xi3[0],n=1,a=1)
            ds=dsdist_calculation(xi2_i,r,px_g,py_g,pz_g,num=-1)

        coords_theor,s_theor,r_g, px_g, py_g, pz_g=make_gencurves(s,r,px_g, py_g, pz_g,xi1_i,xi2_i,xi3_i,dphi,r0,E_g)

        ##
        ##Discretize a theoretical model
        ##
        """
        vox_state
            0：empty
            1：softbody
            2：shell
            11：aperture_softbody
            12：aperture_shell

        vox_state_last:array to store the voxel state of aperture
        """
        #s_count_ini=theta[-1]-0.5*np.pi #Search for a range of pi/2-5/2pi from aperture for hollow section
        #s_count_ini=s_theor[-1]-0.5*np.pi 
        #s_count_end=s_count_ini-2*np.pi
        
        l_count_ini=cvt_theta2l(theta[-1]-0.5*np.pi,Wr,Tr, Dr, r0)
        l_count_end=cvt_theta2l(theta[-1]-5*np.pi/2,Wr,Tr, Dr, r0)
        
        s_count_ini=np.log(1+(E_g*l_count_ini/r0))/E_g
        s_count_end=np.log(1+(E_g*l_count_end/r0))/E_g
        
        
        roop_count_list=[]
        len_unique_s=len(np.unique(s_theor))
        index_list=[]
        
        for j in np.unique(s_theor):
            
            roop_count_list.append(j)
            if len(roop_count_list)==1 or len(roop_count_list)==len_unique_s/5 or len(roop_count_list)==len_unique_s/2 or len(roop_count_list)==len_unique_s/5*4:
                print("roop",len(roop_count_list),"/",len_unique_s)
            
            coords_aperture=coords_theor[s_theor==j]
            coords_genecurves=np.array([px_g, py_g, pz_g]).T[s_theor==j]
            coords_r=r_g[s_theor==j]
            len_coords_aperture=len(coords_aperture)

            ##
            ##KDTree
            ##
            tree = KDTree(np.array([vox_coords[0].ravel(),vox_coords[1].ravel(),vox_coords[2].ravel()]).T)
            
            if j == s_theor[-1]:
                
                for k in range(0,len_coords_aperture):
                    apgen_dist=distance.euclidean(coords_aperture[k],coords_genecurves[k])
                    target = coords_aperture[k]
                    dist, index = tree.query(target)
                    
                    if apgen_dist <= 0.97*coords_r[k] and vox_state_last.ravel()[index]==0:
                        vox_state.ravel()[index]=11
                        vox_state_last.ravel()[index]=11

                    elif apgen_dist > 0.97*coords_r[k] and vox_state_last.ravel()[index]==0:
                        vox_state.ravel()[index]=12
                        vox_state_last.ravel()[index]=12
                        index_list.append(index)
                        
            else:
                for k in range(0,len_coords_aperture):
                    apgen_dist=distance.euclidean(coords_aperture[k],coords_genecurves[k])
                    target = coords_aperture[k] 
                    dist, index = tree.query(target)
                    
                    if apgen_dist > 0.97*coords_r[k]:
                        if vox_state.ravel()[index]==0:
                            vox_state.ravel()[index]=2
                        if s_count_end < j and j < s_count_ini and vox_state_last.ravel()[index]==0:
                            vox_state_last.ravel()[index]=2  #hollow section:Serch last pi/2~5/2pi 
                           
                    elif apgen_dist <= 0.97*coords_r[k]:
                        if vox_state.ravel()[index]==0:
                            vox_state.ravel()[index]=1
                        if s_count_end < j and j < s_count_ini and vox_state_last.ravel()[index]==0:
                            vox_state_last.ravel()[index]=1  #hollow section:Serch last pi/2~5/2pi 
                            
        ##                    
        ##Prevent shell loss                    
        ##
        
        
        z_max_coord=max([coords_list[o][2] for o in index_list])
        len_x=len(vox_state[:,0,0])
        len_y=len(vox_state[0,:,0])
        len_z=len(vox_state[0,0,:])
        vox_state_copy=np.zeros([len_x,len_y,len_z])

        for m in range(0,len(vox_state.ravel())):
            if vox_state.ravel()[m]==1:
                if vox_state.ravel()[m+1]==0 and coords_list[m][2]<z_max_coord:
                    vox_state_copy.ravel()[m]=2
                    vox_state_copy.ravel()[m+1]=2

                if vox_state.ravel()[m+1]==0 and coords_list[m][2]==z_max_coord:
                    vox_state_copy.ravel()[m]=2
                
                if vox_state.ravel()[m-1]==0:
                    vox_state_copy.ravel()[m]=2
                    vox_state_copy.ravel()[m-1]=2

                if vox_state.ravel()[m+len_z*len_y]==0:
                    vox_state_copy.ravel()[m]=2
                    vox_state_copy.ravel()[m+len_z*len_y]=2

                if vox_state.ravel()[m-len_z*len_y]==0:
                    vox_state_copy.ravel()[m]=2
                    vox_state_copy.ravel()[m-len_z*len_y]=2

                if vox_state.ravel()[m+len_z]==0:
                    vox_state_copy.ravel()[m]=2
                    vox_state_copy.ravel()[m+len_z]=2

                if vox_state.ravel()[m-len_z]==0:
                    vox_state_copy.ravel()[m]=2
                    vox_state_copy.ravel()[m-len_z]=2

        vox_state.ravel()[vox_state_copy.ravel()==2]=2
        vox_state.ravel()[vox_state_last.ravel()==12]=12
        vox_state.ravel()[vox_state_last.ravel()==11]=11

        volume_total=np.count_nonzero(vox_state!=0)
        volume_shell=np.count_nonzero((vox_state==2)|(vox_state==12))
        volume_softtissue=np.count_nonzero((vox_state==1)|(vox_state==11))
        softtissue_rate=volume_softtissue/volume_total

        #filename="T"+str(ret[i,0])+"delta"+str(ret[i,1])
        np.savez(filename,vox_state_last=vox_state_last,dx=dx,vox_coords=vox_coords,coords_list=coords_list,vox_state=vox_state,softtissue_rate=softtissue_rate)
        
    pass

## Parallelization

In [16]:
def parallel(i):
    filename=str(i)
    result=generatemodel(theta,Wr,ret[i,0],Dr,Sr,ret[i,1],gamr,r0,Phi,l_step0,vox_reso,filename)
    return result

## Implementation

In [17]:
t1=time.time()
result=joblib.Parallel(n_jobs=6,verbose=5)(joblib.delayed(parallel)(i) for i in range(0,len(ret)))#130,176

t2=time.time()
print(t1-t2)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed: 74.8min
C:\Users\araki\miniconda3\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed: 501.2min
[Parallel(n_jobs=6)]: Done 276 tasks      | elapsed: 2261.2min


-289404.17744255066


[Parallel(n_jobs=6)]: Done 400 out of 400 | elapsed: 4823.4min finished
